In [32]:
import requests, sqlalchemy
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import pandas as pd
from tqdm import tqdm

headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}

In [44]:
def scanner(subject):
    
    # store sites as a dictionary
    # key is site name, value is a list - element 1 is the search url, element 2 is the search result element, and 3 is the class
    sites = {"aljazeera": [f"https://www.aljazeera.com/search/{subject}", "a", "u-clickable-card__link"]}
    
    
    articles = {}
    df = pd.DataFrame()
    article_sites = []
    titles = []
    article_contents = []

    print("Scanning...\n")
    for site in tqdm(sites.items()):
        article_links = []
        print(site)
        
        r = requests.get(site[1][0], headers=headers)
        c = r.content
        soup = BeautifulSoup(c, "html.parser")
        page_results = soup.find_all(site[1][1], {"class": site[1][2]})
        for page in tqdm(page_results):
            print(page['href'])
            article_links.append(str(page['href']))
            
            for article in article_links:
                r = requests.get(article, headers=headers)
                c = r.content 
                soup = BeautifulSoup(c, "html.parser")
                page_title = soup.select_one(".article-header").getText()
                page_content = soup.select_one(".l-col").getText()
                article_sites.append(site[0])
                titles.append(page_title)
                article_contents.append(page_content)
    
    df['source'] = [site for site in article_sites]
    df['title'] = [title for title in titles]
    df['content'] = [content for content in article_contents]
    return df

In [45]:
df = scanner("COP26")
df.head()

Scanning...



  0%|          | 0/1 [00:00<?, ?it/s]

('aljazeera', ['https://www.aljazeera.com/search/COP26', 'a', 'u-clickable-card__link'])


https://www.aljazeera.com/news/2021/9/6/cop26-summit-urged-to-prioritise-adaptation-as-emergency-surges


https://www.aljazeera.com/news/2021/10/31/cop26-begins-with-urgent-calls-to-avert-climate-disaster


...

https://www.aljazeera.com/news/2021/11/12/deforestation-in-brazils-amazon-rising-despite-cop26-pledges


https://www.aljazeera.com/news/2021/10/28/infographic-cop26-who-is-attending


https://www.aljazeera.com/news/2021/11/8/obama-hits-russia-china-for-lack-of-urgency-on-climate


100%|██████████| 1/1 [02:41<00:00, 161.70s/it]


,source,title,content
0,aljazeera,News|Climate CrisisCOP26 urged to prioritise a...,News|Climate CrisisCOP26 urged to prioritise a...
1,aljazeera,News|Climate CrisisCOP26 urged to prioritise a...,News|Climate CrisisCOP26 urged to prioritise a...
2,aljazeera,News|Climate CrisisCOP26: World gathers amid c...,News|Climate CrisisCOP26: World gathers amid c...
3,aljazeera,News|Climate CrisisCOP26 urged to prioritise a...,News|Climate CrisisCOP26 urged to prioritise a...
4,aljazeera,News|Climate CrisisCOP26: World gathers amid c...,News|Climate CrisisCOP26: World gathers amid c...


In [49]:
df['title'][0]

'News|Climate CrisisCOP26 urged to prioritise adaptation as climate emergency surgesMore than 50 officials convening in Rotterdam call for November’s COP26 climate talks to treat adaptation as ‘urgent’.'

In [50]:
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///sent_data.db')
df.to_sql("news_data", engine, if_exists="append", index=False)